In [1]:
import scapy
from scapy.all import sniff, get_if_list, conf, Scapy_Exception
from collections import Counter
import pandas as pd
import socket
import os

/Users/joshholt/opt/anaconda3/lib/python3.9/site-packages/scapy/layers/ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/Users/joshholt/opt/anaconda3/lib/python3.9/site-packages/scapy/layers/ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


In [2]:
pd.options.display.width = 100

In [3]:
conf.promisc = 1

In [4]:
print(conf.promisc)

1


In [5]:
#print(conf)

In [6]:
def get_packets(interface = "ethernet_0", filename = "sniff_cap.csv", count = 100, print_status = False, timeout=None):
    """
    This function is designed to capture a number of ip packets to analyize with pandas.
    """
    protocol_numbers = {1: "ICMP", 2: "IGMP", 6: "TCP", 17: "UDP", 41: "IPv6",
                        47: "GRE", 50: "ESP", 51: "AH", 58: "ICMPv6", 89: "OSPF",
                        103: "PIM", 132: "SCTP"}

    #Checks if file exists
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        return df
    #Sniffs on all interfaces for 'count' number of ip packets
    else:    
        sniffed_packets = sniff(iface=interface, count=count, filter="ip", timeout=timeout)
        list_of_packets = []
        i = 0

        for packet in sniffed_packets:
            if packet.haslayer('IP'):
                try:
                    source_name = socket.gethostbyaddr(packet['IP'].src)
                except socket.herror:
                    source_name = None
                try:
                    destination_name = socket.gethostbyaddr(packet['IP'].dst)
                except socket.herror:
                    destination_name = None
                i += 1
                #Keeps track of how many captures and prints to screen
                if print_status == True:
                    if i % (count/10) == 0:
                        print(f"Percent complete: {(i/count) * 100} %")

                #Collects the information into a dictionary for later use in Pandas
                protocol_number = packet['IP'].proto
                
                temp_dict = {"protocol_name": protocol_numbers.get(protocol_number, "Unknown"),
                             "src_IP" : str(packet["IP"].src),
                             "src_name": source_name,
                             "dst_IP" : str(packet["IP"].dst),
                             "dst_name": destination_name,
                             "payload": str(packet["IP"].payload)}
                if packet.haslayer('TCP'):
                    temp_dict["source_port"] = str(packet["TCP"].sport)
                    temp_dict["dst_port"] = str(packet["TCP"].dport)
                    
                elif packet.haslayer('UDP'):
                    temp_dict["source_port"] = str(packet["UDP"].sport)
                    temp_dict["dst_port"] = str(packet["UDP"].dport)

                else:
                    temp_dict["source_port"] = None
                    temp_dict["dst_port"] = None
                list_of_packets.append(temp_dict)
                
        df = pd.DataFrame(list_of_packets)
        df.to_csv(filename, index=False)

        return df

In [7]:
list_of_ifaces0 = get_if_list()

In [8]:
#print(list_of_ifaces0)

In [9]:
usable_ifaces = []

In [ ]:
for iface in list_of_ifaces0:
    try:
        iface = iface.replace("{", "").replace("}", "")
        #print(iface)
        sniff(iface=iface, count=5, timeout=10)
        usable_ifaces.append(iface)
    except (OSError, Scapy_Exception) as e:  # Catch both OSError and Scapy_Exception
        print(f"Error with interface {iface}: {e}")
        pass

In [ ]:
#print(usable_ifaces)

In [ ]:
i = 0
for item in usable_ifaces:
    sniffer = get_packets(interface=str(item), filename = f"sniff_{i}.csv", timeout=60)
    #print(sniffer['src_port'].value_counts().head(3))
    #print(sniffer['dst_port'].value_counts().head(3))
    print(f"Interface: {item} and sniff number: {i} with {len(sniffer)} packets.")
    print("-----------------------------------------------------------")
    i+= 1
print(f"Usable interfaces: {i}")

In [ ]:
#df.shape